<a href="https://colab.research.google.com/github/ashwath-tech/llama-3.2-grumpy-it-finetune/blob/main/Training/FineTune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -------------------SET UP-------------------------
# get the secret key from huggingface
# put it in the secrets section and name it HF_TOKEN
# --------------------------------------------------

In [1]:
# Cleanup and Install
# We force-reinstall torch and torchvision together to ensure they match
!pip uninstall -y torch torchvision torchaudio transformers peft trl
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers peft datasets bitsandbytes accelerate
print("NOW RESTART THE RUNTIME (Runtime > Restart Session) BEFORE CONTINUING!")

In [3]:
#imports
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from google.colab import userdata
from huggingface_hub import login


In [4]:
#login huggingface
try:
    hf_token = userdata.get('HF_TOKEN')
    login(hf_token, add_to_git_credential=True)
except:
    login()

In [ ]:
#get model, tokenizer and quantize
model_name = "meta-llama/Llama-3.2-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
#preperation
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

dataset = load_dataset("NotSure123/grumpy-it-dataset", split="train")

In [ ]:
def format_and_tokenize(examples):

    texts = [
        tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=False)
        for msgs in examples["messages"]
    ]

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=1024,
        padding="max_length",
    )

    input_ids = tokenized["input_ids"]
    labels = []
    for seq in input_ids:
        seq_labels = [
            -100 if token == tokenizer.pad_token_id else token
            for token in seq
        ]
        labels.append(seq_labels)

    tokenized["labels"] = labels
    return tokenized
tokenized_dataset = dataset.map(format_and_tokenize, batched=True)


In [8]:
#set-up trainer
training_args = TrainingArguments(
    output_dir="./grumpy_llama_results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    logging_steps=1,
    max_steps=80,
    optim="paged_adamw_32bit",
    fp16=True,
    save_strategy="steps",
    save_steps=20,
    report_to="none"
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

In [ ]:
#training
trainer.train()

In [ ]:
#upload to huggingface
repo_name = "XXXXXX/grumpy-llama-3.2-3B" #Your repo name
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)